<a href="https://colab.research.google.com/github/KPsimo/Kidus_Football_Games/blob/main/DSRP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pandas

import warnings
warnings.filterwarnings('ignore')



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score

In [23]:
# fixed_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/games.csv')
# fixed_df.isnull()
from google.colab import drive
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/My Drive Colab Notebooks /games.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
df.columns = df.columns.str.title()
df.columns = df.columns.str.replace('_', ' ')

In [25]:
df['Temp'].median()

59.0

In [26]:
df[df['Wind']>55]
df = df.drop(index =[2396])
df= df.drop(index =[4703])

In [27]:
df[df['Result']<-20]
df.columns


Index(['Game Id', 'Season', 'Game Type', 'Week', 'Gameday', 'Weekday',
       'Gametime', 'Away Team', 'Away Score', 'Home Team', 'Home Score',
       'Location', 'Result', 'Total', 'Overtime', 'Old Game Id', 'Gsis',
       'Nfl Detail Id', 'Pfr', 'Pff', 'Espn', 'Ftn', 'Away Rest', 'Home Rest',
       'Away Moneyline', 'Home Moneyline', 'Spread Line', 'Away Spread Odds',
       'Home Spread Odds', 'Total Line', 'Under Odds', 'Over Odds', 'Div Game',
       'Roof', 'Surface', 'Temp', 'Wind', 'Away Qb Id', 'Home Qb Id',
       'Away Qb Name', 'Home Qb Name', 'Away Coach', 'Home Coach', 'Referee',
       'Stadium Id', 'Stadium'],
      dtype='object')

Removing unnessecary columns

In [35]:
cols_to_drop = ['Away Qb Name', 'Home Qb Name', 'Away Qb Id', 'Home Qb Id', 'Away Coach',
               'Home Coach', 'Pff', 'Ftn', 'Away Rest', 'Home Rest', 'Pff', 'Ftn', 'Nfl Detail Id',
               'Pfr', 'Espn', 'Away Rest', 'Home Rest', 'Old Game Id', 'Gsis', 'Stadium Id', 'Stadium',
               'Under Odds', 'Over Odds', 'Div Game', 'Game Id']
fixed_df = df.drop(columns=cols_to_drop)
fixed_df.columns

Index(['Season', 'Game Type', 'Week', 'Gameday', 'Weekday', 'Gametime',
       'Away Team', 'Away Score', 'Home Team', 'Home Score', 'Location',
       'Result', 'Total', 'Overtime', 'Away Moneyline', 'Home Moneyline',
       'Spread Line', 'Away Spread Odds', 'Home Spread Odds', 'Total Line',
       'Roof', 'Surface', 'Temp', 'Wind', 'Referee'],
      dtype='object')

In [36]:
fixed_df.loc[fixed_df['Roof'] == 'closed', 'Temp'] = fixed_df.loc[fixed_df['Roof'] == 'closed', 'Temp'].fillna(fixed_df['Temp'].median())
fixed_df.loc[fixed_df['Roof'] == 'dome', 'Temp'] = fixed_df.loc[fixed_df['Roof'] == 'dome', 'Temp'].fillna(fixed_df['Temp'].median()) #Regulated temp so I took the median to balalnce

In [37]:
fixed_df.loc[fixed_df['Roof'] == 'closed', 'Wind'] = fixed_df.loc[fixed_df['Roof'] == 'closed', 'Wind'].fillna(0)
fixed_df.loc[fixed_df['Roof'] == 'dome', 'Wind'] = fixed_df.loc[fixed_df['Roof'] == 'dome', 'Wind'].fillna(0) #Indoors has no wind
pd.set_option('display.max_columns', None)  # Show all columns

In [39]:
# categories = ['PLACEHOLDER']
# bars = fixed_df[''].value_counts()

# plt.bar(categories, bars)
# plt.show()

In [40]:
print('a')

a
